# From OMOP to IDEA4RC Cancer Episode

In [ ]:
import mysql.connector
import pandas as pd
import psycopg2
import random
import numpy as np
from datetime import datetime, timedelta

In [ ]:
conn = psycopg2.connect(
    dbname="omopdb",
    user="postgres",
    password="mysecretpassword",
    host="localhost",
    port="5432"
)

curOMOP = conn.cursor()

In [ ]:
config = {
    'user': 'user', 
    'password': 'password',
    'host': '127.0.0.1',
    'database': 'idea4rc_dm',
    'raise_on_warnings': True
}

conn2 = mysql.connector.connect(**config)
curIDEA = conn2.cursor()

# Doubts:

How is the hash supposed to be created so it hashes to the same cancer episodes, with wrong diagnosis? how do we know if they are the same with OMOP?

Version?

### From Procedure Occurrence to Cancer Episode

In [ ]:
query= """
    SELECT id
    FROM patient;
"""

query2= """
    SELECT procedure_date, procedure_concept_id, provider_id 
    FROM procedure_occurrence WHERE patient_id = %s
    AND procedure_concept_id IN (4171863,4321878,4321986,4228202,4279903);
"""
curOMOP.execute(query)

results = curIDEA.fetchall()

for row in results:
    curIDEA.execute(query2, (row))
    results2 = curIDEA.fetchall()
    for row2 in results2:
        query_insert= """
            INSERT INTO cancer_episode (patient_id, procedure_date, procedure_concept_id, provider_id)
            VALUES (%s, %s, %s, %s)
        """
        curIDEA.execute(query_insert, (row[0], row2[0], row2[1], row2[2]))

### From Condition Occurrence to Cancer Episode

for histology_adenocarcinoma, what should the code be for 'LOBULAR_CARCINOMA','SALIVARY_DUCT_CARCINOMA','MAMMARY_SECRETORY_CARCINOMA','MYOEPITHELIAL_CARCINOMA','EX_PLEOMORPHIC_ADENOCARCINOMA','SEBACEOUS_LYMPHADENOCARCINOMA', 'SALIVARY_GLAND','NASOPHARYNX_PAPILLARY_ADENOCARCINOMA'???. Also we have 10 codes which we don't find in the initdb.sql

histology_neuroendocrine we have 2 codes we dont find in the initdb.sql

histology_odontogenic_carcinoma 1 code we dont find in the db, and 'ODONTOGENIC_CARCINOMA','AMELOBLASTIC_CARCINOMA','DEDIFFERENTIATED_INTRAOSSEOUS_AMELOBLASTIC_CARCINOMA','DEDIFFERENTIATED_PERIPHERAL_AMELOBLASTIC_CARCINOMA','PRIMARY_INTRAOSSEOUS_CARCINOMA','INTRAOSSEOUS_CARCINOMA_DEVELOPED_ON_ODONTOGENIC_CYST'

I believe these columns are missing: Histology subgroup Adenosquamous carcinoma, Histology subgroup Teratocarcinosarcoma, Histology subgroup NUT carcinoma, Histology subgroup HPV-related Multiphenotypic Sinonasal Carcinoma, Histology subgroup Olfactory neuroblastoma (esthesioneuroblastoma; esthesioneurocytoma; esthesioneuroepithelioma; olfactory placode tumor), 




In [ ]:
dictionary = {
    'histology_squamous':[4078953, 4166826, 4206785, 4009590, 4247661, 4029973, 37156145, 4312219, 4052146, 
4175678, 4233949, 4191609, 37156145, 37204531, 37204532],
    'histology_adenocarcinoma':[4097305, 37152526, 4050978, 4247921, 4238334, 4172953, 4022895, 4253608, 
4030121, 4164740, 4148292, 4146684, 4221403, 4224593, 4323699, 4029680, 
4321002, 4066512, 4182993, 4271564, 4212379, 4050978],
    'histology_small_cell':[4307588, 4307589, 4307590, 4307591, 4307592, 4307593, 4307594, 4307595, 4307596]
}

histology_squamous = {
    4078953: 'KERATINIZING_SQUAMOUS_CELL_CARCINOMA',
    4166826: 'NON_KERATINIZING_SQUAMOUS_CELL_CARCINOMA',
    4206785: 'SCHNEIDERIAN_CARCINOMA',
    4009590: 'SPINDLE_CELLSQUAMOUS_CELL_CARCINOMA',
    4247661: 'LYMPHOEPITHELIAL_CARCINOMA',
    4029973: 'BASALOID_SQUAMOUS_CELL_CARCINOMA',
    4233949: 'VERRUCOUS_SQUAMOUS_CELL_CARCINOMA',
    4191609: 'PAPILLARY_SQUAMOUS_CELL_CARCINOMA',
    37156145: 'SQUAMOUS_CELL_CARCINOMA',
    37204531: 'SQUAMOUS_CELL_CARCINOMA_HPV_POSITIVE',
    37204532: 'SQUAMOUS_CELL_CARCINOMA_HPV_NEGATIVE'
}

histology_adenocarcinoma = {
    4097305: '???',
    37152526: '???',
    4050978: 'CYSTOADENOCARCINOMA',
    4247921: 'MUCINOUS',
    4238334: 'CERUMINOUS',
    4172953: '???',
    4022895: 'ADENOID_CYSTIC_CARCINOMA',
    4253608: 'MUCOEPIDERMOID_CARCINOMA',
    4030121: 'POLYMORPHOUS',
    4164740: 'ACINAR_CELL_CARCINOMA',
    4148292: 'CLEAR_CELL_CARCINOMA',
    4146684: 'BASAL_CELL_ADENOCARCINOMA',
    4221403: '???',
    4224593: '???',
    4323699: '???',
    4029680: '???',
    4321002: '???',
    4066512: '???',
    4182993: 'SEBACEOUS_ADENOCARCINOMA',
    4271564: 'CARCINOSARCOMA',
    4212379: 'OXYPHILIC_CARCINOMA',
    4050978: '???'
}

histology_neuroendocrine= {
    36714029:'SMCC',
    37018672:'POORLY_DIFFERENTIATED_NEUROENDOCRINE_CARCINOMA',
    4029971:'LARGE_CELL_NEUROENDOCRINE_CARCINOMA',
    4133828:'???',
    4133297:'???'
}
histology_odontogenic_carcinoma= {
    4098585:'???',
    37116966:'CLEAR_CELL_ODONTOGENIC_CARCINOMA',
    4005818: 'GHOST_CELL_ODONTOGENIC_CARCINOMA'
}

histology_snuc= {
    4243327:'SNUC'
}



